In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install -qU langchain-openai

# 构建一个检索增强生成 (RAG) 应用
RAG是一种通过额外数据增强大型语言模型知识的技术。

大型语言模型可以推理广泛的主题，但它们的知识仅限于训练时使用的特定时间点的公共数据。如果您想构建能够推理私有数据或在模型截止日期后引入的数据的AI应用程序，您需要用模型所需的特定信息来增强模型的知识。将适当的信息引入并插入到模型提示中的过程称为检索增强生成（RAG）。

LangChain有许多组件旨在帮助构建问答应用程序，以及更一般的RAG应用程序。

注意：在这里我们专注于非结构化数据的问答。如果您对结构化数据的RAG感兴趣，请查看我们关于SQL数据的问答的教程。

## 概念
一个典型的RAG应用程序有两个主要组件：

索引：一个从源数据中摄取数据并进行索引的管道。这通常在离线进行。

检索和生成：实际的RAG链，在运行时接收用户查询并从索引中检索相关数据，然后将其传递给模型。

从原始数据到答案的最常见完整序列如下：

## 索引
加载: 首先我们需要加载我们的数据。这是通过 文档加载器 完成的。
分割: 文本分割器 将大型 文档 分割成较小的块。这对于索引数据和将其传递给模型都很有用，因为大型块更难搜索，并且无法适应模型的有限上下文窗口。
存储: 我们需要一个地方来存储和索引我们的分割，以便后续可以进行搜索。这通常使用 向量存储 和 嵌入模型 来完成。

## 检索与生成
检索: 根据用户输入，从存储中使用 检索器 检索相关的分割。
生成: 聊天模型 / 大型语言模型 使用包含问题和检索数据的提示生成答案。

In [2]:
# 从电脑环境变量中导入
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
print("OPENAI_API_KEY:", OPENAI_API_KEY is not None)
print("LANGCHAIN_API_KEY:", LANGCHAIN_API_KEY is not None)

# 设置环境变量以启用LangChain的跟踪功能
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# 设置项目名称以便在LangChain跟踪界面中区分不同项目
os.environ["LANGCHAIN_PROJECT"] = "llm-langchain-learn"

# 兼容 DashScope：将同一个 Key 同步给 DASHSCOPE_API_KEY（若你单独配置了，可删掉此行）
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["DASHSCOPE_API_KEY"] = OPENAI_API_KEY

OPENAI_API_KEY: True
LANGCHAIN_API_KEY: True


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=OPENAI_API_KEY,
    model="qwen-plus"
)

In [4]:
# 我们将构建一个应用程序，回答有关网站内容的问题。
import bs4
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=DashScopeEmbeddings()  # 从环境变量 DASHSCOPE_API_KEY 读取 Key
)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

# 直接在本地构造 RAG 提示，避免依赖 langchain.hub
prompt = ChatPromptTemplate.from_template(
    """
You are a helpful assistant. Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know.

Context:
{context}

Question:
{question}

Helpful Answer:
"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

USER_AGENT environment variable not set, consider setting it to identify your requests.


ModuleNotFoundError: No module named 'dashscope'